# Ratterdam Place Field Repetition
# Temporal Decoding 

In [1]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
from scipy.stats import sem
import utility_fx as util
import os
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import ratterdam_RepetitionCoreFx as RepCore
import RateMapClass_William_20190308 as RateMapClass
import williamDefaults as wmDef
from matplotlib import path
from matplotlib.backends.backend_pdf import PdfPages
import more_itertools, itertools
from sklearn.metrics import auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from scipy.interpolate import splrep, splev
from scipy.spatial import ConvexHull
import scipy

In [2]:
%qtconsole --style native
%matplotlib qt5

In [240]:
# Take each cell in a day and generate a plot with rate map overlaid w detected field edges
# and field dynamics (Fr over time) below
rat = "R808"
day = "D6"
savepath = f'E:\\Ratterdam\\{rat}\\ratterdam_plots\\{day}\\'
df = f'E:\Ratterdam\\{rat}\\{rat}_RatterdamOpen_{day}\\'
clust = 'TT15\\cl-maze1.1'
p, s = loadRepeatingUnit(df, clust)
unit = Unit(s,p, clust)

0.034339063273122784
0.10316680101289039
0.6842628520340124


E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in double

In [3]:
def shuffleFields(unit):
    """
    Takes a unit class as input and shuffles the firing rates of each field
    w.r.t. time. Returns a list of (n,2) numpy arrays where n is the number of
    visits for each field (and thus n varies as a fx of field)
    """
    fields = []
    for field in unit.fields:
        fields.append(np.column_stack((field[:,0],np.random.permutation(field[:,1]))))
    return fields

## k-Nearest Neighbor Temporal Analysis
### For each point, find n nearest neighbors in vector space and find mean actual distance in the array
### Then shuffle the subfield visits, regenerate (null) semaphore plots and repeat (important not to just shuffle semaphores bc sliding window contamination)

In [241]:
# Create real and 1 null semaphore and plot, just to look
diffmats = makeSemaphores(unit.fields)

sfields = shuffleFields(unit)
smats = makeSemaphores(sfields)

plotMats(diffmats,shuff=False)
plotMats(smats,shuff=True)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\colorbar.py:215: UserWarning: Use the colorbar set_ticks() method instead.
  warnings.warn("Use the colorbar set_ticks() method instead.")


In [242]:
# kNN analysis
ns = 1000 # num shuffles
# real
flatmats = [i.flatten() for i in diffmats]
neigh = NearestNeighbors(n_neighbors=2).fit(flatmats)
d,idx = neigh.kneighbors(flatmats)
meanDist = np.mean(np.abs(np.diff(idx)))
meanvecDist = np.mean(d[:,1])

# shuffle
shuffmeanIdxDists = []
shuffmeanVecDists = []
for n in range(ns):
    sfields = shuffleFields(unit)
    smats = makeSemaphores(sfields)
    flatsmats = [i.flatten() for i in smats]
    sneigh = NearestNeighbors(n_neighbors=2).fit(flatsmats)
    sd,sidx = sneigh.kneighbors(flatsmats)
    smeanDist = np.mean(np.abs(np.diff(sidx)))
    shuffmeanDists.append(smeanDist)
    smeanvecDist = np.mean(sd[:,1])
    if smeanvecDist < 1000: # some shuff values are like 10e19. Maybe related to the 'impossible result' runtime error?
        shuffmeanVecDists.append(smeanvecDist)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-pac

In [243]:
plt.figure()
plt.hist(shuffmeanDists,bins=15)
plt.vlines(meanDist,0,160,'r')
np.percentile(shuffmeanDists,5)
null5pct = np.percentile(shuffmeanDists,5)
plt.vlines(null5pct,0,160,'k')
plt.title("Average Index Distance kNN vs Shuffle")

plt.figure()
plt.hist(shuffmeanVecDists,bins=100)
plt.vlines(meanvecDist,0,160,'r')
np.percentile(shuffmeanVecDists,5)
null5pct = np.percentile(shuffmeanVecDists,5)
plt.vlines(null5pct,0,160,'k')
plt.title("Average Vector Distance kNN vs Shuffle")

Text(0.5,1,'Average Vector Distance kNN vs Shuffle')

### kNN Classification - Temporal Epoch
### Divide session into n epochs, try to decode. Standard train/test classification

In [299]:
# Setup parameters and setup data
n = 3
testProp = 0.33
idx = np.ceil(np.linspace(0,len(diffmats),n+1)).astype(np.int)
diffmats = makeSemaphores(unit.fields)
flatmats = np.asarray([i.flatten() for i in diffmats])
split = np.split(flatmats,idx)
labels = []
for i in range(len(idx)-1):
    l= idx[i+1]-idx[i]
    labels.extend([i]*l)
labels = np.asarray(labels)

In [300]:
# real
rep = 500
perfs = []
for i in range(rep):
    train, test = train_test_split(list(range(len(diffmats))), test_size=testProp)
    Xtrain, Xtest, Ytrain, Ytest = flatmats[train], flatmats[test], labels[train], labels[test]
    neigh = KNeighborsClassifier(n_neighbors=2)
    neigh.fit(Xtrain, Ytrain)
    perfs.append(sum(Ytest==neigh.predict(Xtest))/len(Xtest))  
real5pct = np.percentile(perfs,5)
plt.figure()
plt.hist(perfs)
plt.vlines(real5pct,0,100,'r')
plt.vlines(1/3,0,100,'grey')
plt.title(f"kNN Classifier of Semaphore Plots, {n} Epochs, {rep} runs", fontsize=18)

Text(0.5,1,'kNN Classifier of Semaphore Plots, 3 Epochs, 500 runs')

In [302]:
# shuffle
rep = 500
perfs = []
for i in range(rep):
    sfields = shuffleFields(unit)
    smats = makeSemaphores(sfields)
    flatsmats = np.asarray([i.flatten() for i in smats])
    
    train, test = train_test_split(list(range(len(flatsmats))), test_size=testProp)
    Xtrain, Xtest, Ytrain, Ytest = flatsmats[train], flatsmats[test], labels[train], labels[test]
    neigh = KNeighborsClassifier(n_neighbors=2)
    neigh.fit(Xtrain, Ytrain)
    perfs.append(sum(Ytest==neigh.predict(Xtest))/len(Xtest))  
    
real5pct = np.percentile(perfs,5)
plt.figure()
plt.hist(perfs)
plt.vlines(real5pct,0,100,'r')
plt.vlines(1/3,0,100,'grey')
plt.title(f"kNN Classifier of Semaphore Plots, {n} Epochs, {rep} runs, shuff visits within field", fontsize=18)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))


Text(0.5,1,'kNN Classifier of Semaphore Plots, 3 Epochs, 500 runs, shuff')

## kNN Classification - Analysing Any Cooperative Temporal Decoding between Fields within a cell
### Decode temporal information (in epoch) from real data
### Use 1) fields exchanged between units (matched to # real fields/cell) and 2) rotational shuffling within cell
### versus shuffling visits within fields for each cell. This allows a dissociation between temporal decoding that is just from
### single fields acting independently versus some 'cooperartive' coding between real fields for a repeating PC

In [34]:
# create a list of fields - all the fields from all the units. randomly grab from this
allfields = []
for u in population.values():
    allfields.extend(u.fields)

In [36]:
class GuttedUnit():
    
    def __init__(self):
        self.fields = []

In [37]:
def createShuffledUnit(unit, allfields):
    """
    Input - unit: a unit object. unit.fields is a list of (n,2) arrays. Each entry is [ts,fr] where ts is in min and fr is in Hz
          - allfields: a list of all fields from the population of repeating units. Each entry is a (n,2) array. Each entry is [ts,fr] where ts is in min and fr is in Hz          
    This function will take a unit object and draw randomly from all the fields from all rep cells that day and assign to a 'field shuffled unit' (matching the # real fields / cell)   
    Return - a stripped down unit object (only has fields as attribute) with the scrambled fields
    """
    selectedFields = [allfields[i] for i in np.random.randint(0,len(allfields), len(unit.fields))]
    sunit = GuttedUnit()
    sunit.fields = selectedFields
    return sunit

In [249]:
# Setup parameters and setup data
clust = 'TT15\\cl-maze1.1'
unit = population[clust]
epochs = 3 # number of temporal epochs to decode
nshuffles = 1000 # shuffles creating new data each time
nreps = 100 # repeated decoding given a set of data
trainProp = 0.67 # test is the complement
nNeighbors = 2

In [250]:
# real
diffmats = makeSemaphores(unit.fields)
flatmats = np.asarray([i.flatten() for i in diffmats])
idx = np.ceil(np.linspace(0,len(diffmats),epochs+1)).astype(np.int)
labels = []
for i in range(len(idx)-1):
    l= idx[i+1]-idx[i]
    labels.extend([i]*l)
labels = np.asarray(labels)

rperfs = []
for i in range(nreps):
    train, test = train_test_split(list(range(len(diffmats))), train_size=trainProp)
    Xtrain, Xtest, Ytrain, Ytest = flatmats[train], flatmats[test], labels[train], labels[test]
    neigh = KNeighborsClassifier(n_neighbors=nNeighbors)
    neigh.fit(Xtrain, Ytrain)
    rperfs.append(sum(Ytest==neigh.predict(Xtest))/len(Xtest))  
plt.figure()
plt.hist(rperfs)
# real5pct = np.percentile(rperfs,5)
# plt.vlines(real5pct,0,100,'r')
# plt.vlines(1/3,0,100,'grey')
plt.title(f"kNN Classifier of Semaphore Plots, {epochs} Epochs, {nreps} runs, real", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

(array([ 0.,  5., 10., 15., 20., 25., 30.]),
 <a list of 7 Text yticklabel objects>)

In [67]:
# shuffled fields between units (each field real, ownership is shuffled)
sfperfs = []
for r in range(nshuffles):
    sunit = createShuffledUnit(population[clust], allfields)
    diffmats = makeSemaphores(sunit.fields)
    flatmats = np.asarray([i.flatten() for i in diffmats])
    
    train, test = train_test_split(list(range(len(diffmats))), test_size=testProp)
    Xtrain, Xtest, Ytrain, Ytest = flatmats[train], flatmats[test], labels[train], labels[test]
    neigh = KNeighborsClassifier(n_neighbors=2)
    neigh.fit(Xtrain, Ytrain)
    sfperfs.append(sum(Ytest==neigh.predict(Xtest))/len(Xtest))  
    
real5pct = np.percentile(sfperfs,5)
plt.figure()
plt.hist(sfperfs)
plt.vlines(real5pct,0,100,'r')
plt.vlines(1/3,0,100,'grey')
plt.title(f"kNN Classifier of Semaphore Plots, {n} Epochs, {nshuffles} runs, Shuff Fields Between Cells", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

(array([  0.,  50., 100., 150., 200., 250.]),
 <a list of 6 Text yticklabel objects>)

In [73]:
# shuffle visits within fields for a neuron
svperfs = []
for r in range(nshuffles):
    sfields = shuffleFields(unit)
    smats = makeSemaphores(sfields)
    flatsmats = np.asarray([i.flatten() for i in smats])
    
    train, test = train_test_split(list(range(len(flatsmats))), test_size=testProp)
    Xtrain, Xtest, Ytrain, Ytest = flatsmats[train], flatsmats[test], labels[train], labels[test]
    neigh = KNeighborsClassifier(n_neighbors=2)
    neigh.fit(Xtrain, Ytrain)
    svperfs.append(sum(Ytest==neigh.predict(Xtest))/len(Xtest))  
    
real5pct = np.percentile(svperfs,5)
plt.figure()
plt.hist(svperfs)
plt.vlines(real5pct,0,100,'r')
plt.vlines(1/3,0,100,'grey')
plt.title(f"kNN Classifier of Semaphore Plots, {n} Epochs, {nshuffles} runs, shuff visits within field", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-pac

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:512: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\interpolate\

(array([   0., 1000., 2000., 3000., 4000., 5000., 6000., 7000., 8000.]),
 <a list of 9 Text yticklabel objects>)

In [133]:
def rotateFields(unit):
    """
    Input   - unit: Unit class object
            
    selects a temporal shift for each field. Shift is rotational such that 
    whatever 'falls off' end of vector 'comes back' to the front.
    
    Returns - GuttedField object with shifted fields in gunit.fields attribute
    """
    gunit = GuttedUnit()
    sfields = []
    for field in unit.fields:
        nvisits = field.shape[0]
        shift = np.random.randint(0,nvisits)
        sf = np.column_stack((field[:,0], np.roll(field[:,1], shift)))
        sfields.append(sf)
    gunit.fields = sfields
    return gunit

In [169]:
def kNN_semaphore_decoding(unit, reps=50, n=2, epochs=3, stat='median', trainProp=0.67):
    """
    kNearest Neighbor Decoding
    input:  data - Unit class object 
            reps -  # decoding repeats
            n - num neighbors for kNN
            epochs - number of temporal epochs to divide data into to decode
            stat - measure of decoding perf for each set of decoding runs (# = reps).
                    choices are 'median' or 'mean'
            trainProp - proportion of data to use as train (1-trainProp is test)
                    
    Takes Unit class object. Creates series of semaphore plots using makeSemaphores()
    Then decodes temporal information from these. Semaphores divided into epochs,
    ntrain proportion used to find nearest neighbors of test in euclidian space and see
    if epoch labels match. Do multiple times and report a statistic of perf
    
    returns: scalar of decoding perf, as defined by 'stat'
    """
    mats = makeSemaphores(unit.fields)
    flatmats = np.asarray([i.flatten() for i in mats])
    idx = np.ceil(np.linspace(0,len(flatmats),epochs+1)).astype(np.int)
    labels = []
    for i in range(len(idx)-1):
        l= idx[i+1]-idx[i]
        labels.extend([i]*l)
    labels = np.asarray(labels)
    
    perfs = []
    for rep in range(reps):
        train, test = train_test_split(list(range(len(flatmats))), train_size=trainProp)
        Xtrain, Xtest, Ytrain, Ytest = flatmats[train], flatmats[test], labels[train], labels[test]
        neigh = KNeighborsClassifier(n_neighbors=nNeighbors)
        neigh.fit(Xtrain, Ytrain)
        perfs.append(sum(Ytest==neigh.predict(Xtest))/len(Xtest))  
        
    if stat == 'median':
        sumStat = np.median(perfs)
    elif stat == 'mean':
        sumStat = np.mean(perfs)
    else:
        print("Invalid statistic supplied. Choose 'median' or 'mean' ")
        sumStat = None
    
    return sumStat


In [238]:
# rotational shuffle - single cell, attempt to show cooperative temporal coding within cell
# take each field and shift it by a random amount temporally. rotation means whatever data 'falls' off end cycles back to beginning

# edit: below will shift some amount in mins, where each shift is at least a certain dist apart
# but thinking more, maybe you preserve the actual visit times and just shift the activity at each one
# # generate possible time offsets. do it this way to impose a min shift
# minShift = 10/60 # numerator is min shift in s, divide by 60 express as % of a minute
# shifts = np.arange(0,fmax,1/6)

rsperfs = [] # each data point is the median or mean (depending on user choice) of multiple decodings of a single shuffle
for n in range(nshuffles):
    gunit = rotateFields(unit)
    sperf = kNN_semaphore_decoding(gunit, reps=nreps, n=nNeighbors, epochs=epochs, stat='median', trainProp=trainProp)
    rsperfs.append(sperf)

In [239]:
plt.figure()
plt.hist(rsperfs)
rs95pct = np.percentile(rsperfs, 95)
plt.vlines(rs95pct,0,100,'k')
plt.title(f"kNNc Semaphores, {epochs} Epochs, {nshuffles} shuffles, {nreps} clf/shuff, rotat. shuffle", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

(array([  0.,  50., 100., 150., 200., 250., 300., 350., 400., 450.]),
 <a list of 10 Text yticklabel objects>)

### Consolidating rotational shuffle code into blocks/fx
#### Everything but creating the pop dict (and imports) is done here (b/c that part as of mif aug 2020 is still not automated - not using the rep detection alg (though ryl did it) and getting rid of bad fields

In [3]:
class GuttedUnit():   
    def __init__(self):
        self.fields = []

def kNN_semaphore_decoding(fieldArray, reps=50, n=2, epochs=3, stat='median', trainProp=0.67, wnSize=5*1e6*60, wnStep=2*1e6*60):
    """
    kNearest Neighbor Decoding
    input:  data - Unit class object 
            reps -  # decoding repeats
            n - num neighbors for kNN
            epochs - number of temporal epochs to divide data into to decode
            stat - measure of decoding perf for each set of decoding runs (# = reps).
                    choices are 'median' or 'mean'
            trainProp - proportion of data to use as train (1-trainProp is test)
            wnSize - (for semaphore plot) sliding window size to create a single flag plot. unit in mins.
            snStep - (for semaphore plot) sliding window slide amount. unit in mins.
            fieldType - "smoothed" or "raw". Option chooses whether to use smoothed or unsmoothed field attrs of Unit obj
                    
    Takes Unit class object. Creates series of semaphore plots using makeSemaphores()
    Then decodes temporal information from these. Semaphores divided into epochs,
    ntrain proportion used to find nearest neighbors of test in euclidian space and see
    if epoch labels match. Do multiple times and report a statistic of perf
    
    returns: scalar of decoding perf, as defined by 'stat'
    """
        
    mats,_ = RepCore.makeSemaphores(fieldArray, wnSize=wnSize, wnStep=wnStep)
    flatmats = np.asarray([i.flatten() for i in np.asarray(mats)])
    idx = np.ceil(np.linspace(0,len(flatmats),epochs+1)).astype(np.int)
    labels = []
    for i in range(len(idx)-1):
        l= idx[i+1]-idx[i]
        labels.extend([i]*l)
    labels = np.asarray(labels)
    
    perfs = []
    for rep in range(reps):
        train, test = train_test_split(list(range(len(flatmats))), train_size=trainProp)
        Xtrain, Xtest, Ytrain, Ytest = flatmats[train], flatmats[test], labels[train], labels[test]
        neigh = KNeighborsClassifier(n_neighbors=n)
        neigh.fit(Xtrain, Ytrain)
        perfs.append(sum(Ytest==neigh.predict(Xtest))/len(Xtest))  
        
    if stat == 'median':
        sumStat = np.median(perfs)
    elif stat == 'mean':
        sumStat = np.mean(perfs)
    else:
        print("Invalid statistic supplied. Choose 'median' or 'mean' ")
        sumStat = None
    
    return sumStat

def rotateFields(fields):
    """
    Input   - unit: Unit class object      
    selects a temporal shift for each field. Shift is rotational such that 
    whatever 'falls off' end of vector 'comes back' to the front.
    Returns - GuttedField object with shifted fields in gunit.fields attribute
    """
    gunit = GuttedUnit()
    sfields = []
    for field in fields:
        nvisits = field.shape[0]
        shift = np.random.randint(0,nvisits)
        sf = np.column_stack((field[:,0], np.roll(field[:,1], shift)))
        sfields.append(sf)
    gunit.fields = sfields
    return gunit

def shuffleFields(fields):
    """
    Input   - unit: Unit class object   
    Shuffle visits within a field.
    Returns - GuttedField object with shifted fields in gunit.fields attribute
    """
    gunit = GuttedUnit()
    newfields = []
    for field in fields:
        newfields.append(np.column_stack((field[:,0],np.random.permutation(field[:,1]))))
    gunit.fields = newfields
    return gunit

In [100]:
rat = "R808"
day = "D6"
savepath = f'E:\\Ratterdam\\{rat}\\ratterdam_plots\\{day}\\decoding\\'
df = f'E:\Ratterdam\\{rat}\\{rat}_RatterdamOpen_{day}\\'
clustList = util.getClustList(df)
population = {}
for clust in clustList:
    try:
        print(clust)
        unit = RepCore.loadRepeatingUnit(df, clust, smoothing=1)
        rm = util.makeRM(unit.spikes, unit.position)
        if np.nanpercentile(rm, 95) > 1.:
            population[clust] = unit
            print(f"{clust} included")
        else:
            print(f"{clust} is not included")
    except:
        pass

TT12\cl-maze1.1


E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
E:\UserData\Documents\GitHub\ratterdam\ratterdam_DataFiltering.py:127: RuntimeWarning: invalid value encountered in less_equal
  binsBelowThresh = np.where(rm.flatten()<=rateThresh)[0].shape[0]


TT12\cl-maze1.1 is not included
TT12\cl-maze1.10


C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
E:\UserData\Documents\GitHub\ratterdam\ratterdam_RepetitionCoreFx.py:132: RuntimeWarning: invalid value encountered in double_scalars
  field_FR.append(spk.shape[0]/vdur)
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\u

TT12\cl-maze1.10 included
TT12\cl-maze1.2


...

E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)


TT15\cl-maze1.4 is not included
TT15\cl-maze1.5


C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Pyt

TT15\cl-maze1.5 included


E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
E:\UserData\Documents\GitHub\ratterdam\ratterdam_DataFiltering.py:127: RuntimeWarning: invalid value encountered in less_equal
  binsBelowThresh = np.where(rm.flatten()<=rateThresh)[0].shape[0]


In [101]:
# Setup parameters and setup data
parmdict = {
    'epochs':3, # number of temporal epochs to decode
    'nshuffles':1000, # shuffles creating new data each time
    'nreps':10, # repeated decoding given a set of data
    'trainProp':0.4, # test is the complement
    'nNeighbors':2,
    'wnSize':1*1e6*60,
    'wnStep':1*1e6*60,
    'smoothing':1,
    'stat':'median',
    'fieldRateThresh':0.3,
    'fieldpctThresh':20
    
}
data = {}
ts = util.genTimestamp()
for clust in population.keys():
    print(clust)
    try:
        unit = population[clust]
        shuffunit = RepCore.loadRepeatingUnit(df, clust, smoothing=parmdict['smoothing'])


        realperf = kNN_semaphore_decoding(unit.smoothedFields, reps=parmdict['nreps'], n=parmdict['nNeighbors'], epochs=parmdict['epochs'], stat=parmdict['stat'], 
                                          trainProp=parmdict['trainProp'], wnSize=parmdict['wnSize'], wnStep=parmdict['wnStep'])

        rsperfs = [] # each data point is the median or mean (depending on user choice) of multiple decodings of a single shuffle
        vsperfs = []
        for n in range(parmdict['nshuffles']):
            shuffunit.shuffleFieldsFx()
            shuffunit.smoothFieldsFx()

            #rgunit = rotateFields(unit.smoothedFields)
            #sperf = kNN_semaphore_decoding(rgunit.fields, reps=parmdict['nreps'], n=parmdict['nNeighbors'], epochs=parmdict['epochs'], stat=parmdict['stat'], 
               #                           trainProp=parmdict['trainProp'], wnSize=parmdict['wnSize'], wnStep=parmdict['wnStep'])
            #rsperfs.append(sperf)

            sperf = kNN_semaphore_decoding(shuffunit.smoothedFields, reps=parmdict['nreps'], n=parmdict['nNeighbors'], epochs=parmdict['epochs'], stat=parmdict['stat'], 
                                          trainProp=parmdict['trainProp'], wnSize=parmdict['wnSize'], wnStep=parmdict['wnStep'])
            vsperfs.append(sperf)
        if realperf > np.percentile(vsperfs, 95):
            data[unit.name] = {'real':realperf, 'null95':np.percentile(vsperfs, 95), 'pass':True}
        else:
            data[unit.name] = {'real':realperf, 'null95':np.percentile(vsperfs, 95), 'pass':False}
    except:
        print(f"{clust} did not run successfully")

TT12\cl-maze1.10


E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
E:\UserData\Documents\GitHub\ratterdam\ratterdam_RepetitionCoreFx.py:132: RuntimeWarning: invalid value encountered in double_scalars
  field_FR.ap

TT12\cl-maze1.2


E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is

TT12\cl-maze1.3


...

E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
E:\UserData\Documents\GitHub\ratterdam\ratterdam_DataFiltering.py:127: RuntimeWarning: invalid value encountered in less_equal
  binsBelowThresh = np.where(rm.flatten()<=rateThresh)[0].shape[0]


TT15\cl-maze1.3


E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is

TT15\cl-maze1.5


E:\UserData\Documents\GitHub\ratterdam\RateMapClass_William_20190308.py:210: RuntimeWarning: invalid value encountered in greater_equal
  aboveThreshold = np.where(rateMap >= max(fieldThreshold,0), True, False)
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is

In [23]:
plt.figure()
plt.title(unit.name)
#plt.hist(rsperfs,bins=10,color='r')
plt.hist(vsperfs,bins=10,color='k')
plt.vlines(realperf,0,(nshuffles/3)+10, color='red')
plt.vlines(np.percentile(vsperfs, 95),0,nshuffles/3,color='black')
plt.xlabel('Decoding Performance', fontsize=18)
plt.ylabel('Abundance of Decoding Runs', fontsize=18)
#plt.vlines(np.percentile(rsperfs, 95),0,nshuffles/3,color='red')

Text(0,0.5,'Abundance of Decoding Runs')

### get rid of bad fields. Look for % area that has low rate pixels

In [59]:
thresh = 0.3

plt.figure()

for perim in unit.perimeters:
    contour = path.Path(perim)
    spkIn = unit.spikes[contour.contains_points(unit.spikes[:,1:])]
    occIn = unit.position[contour.contains_points(unit.position[:,1:])]
    rm=util.makeRM(spkIn,occIn)
    area = np.sum(~np.isnan(rm.flatten()))
    binsBelowThresh = np.where(rm.flatten()<thresh)[0].shape[0]
    pct = (binsBelowThresh/area)*100
    plt.imshow(rm,aspect='auto',interpolation='None',origin='lower',cmap=cmap,extent=[wmDef.xedges[0], wmDef.xedges[-1], wmDef.yedges[0], wmDef.yedges[-1]])
    plt.plot(perim[:,0], perim[:,1],'.')
    plt.text(perim[0,0],perim[0,1],f"{round(pct,3)}", fontsize=16,color='black', weight='bold')

E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in less
  # This is added back by InteractiveShellApp.init_path()


In [9]:
def filterField(unit, index, rateThresh=0.2, pctThresh=10):
    """
    Inputs: unit - Unit class object
            index - index of field to filter
            rateThresh - pixel rate that will be used to see % pixels below it
            pctThresh -  % of pixels (0-100) with rate below rateThresh above which field should be discarded
    Returns: True/False, as to whether field passes. True = pass
    Function to filter detected place fields by what percent of the field is 'too quiet'. Count pct 
    of pixels with rate below rateThresh. if this pct exceeds pctThresh, return False.
    """
    perim = unit.perimeters[index]
    contour = path.Path(perim)
    spkIn = unit.spikes[contour.contains_points(unit.spikes[:,1:])]
    occIn = unit.position[contour.contains_points(unit.position[:,1:])]
    rm=util.makeRM(spkIn,occIn)
    area = np.sum(~np.isnan(rm.flatten()))
    binsBelowThresh = np.where(rm.flatten()<=rateThresh)[0].shape[0]
    pct = (binsBelowThresh/area)*100
    print(pct)
    if pct > pctThresh:
        return False
    else:
        return True

In [10]:
# filter bad fields from population
for clust in population.keys():
    unit = population[clust]
    filts = [filterField(unit,i,rateThresh=0.3,pctThresh=20) for i in range(len(unit.perimeters))]
    ffields = [unit.fields[i] for i in range(len(unit.fields)) if filts[i] == True]
    fperims = [unit.perimeters[i] for i in range(len(unit.perimeters)) if filts[i] == True]
    unit.fields = ffields
    unit.perimeters = fperims
    unit.smoothFieldsFx()

E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in less_equal


0.0
82.69662921348313
80.38793103448276
0.0
0.0
1.7543859649122806
12.302839116719243
17.424242424242426
66.11570247933885
80.0
0.0
0.0
12.698412698412698
75.38304392236977
49.597855227882036
0.0
0.0
0.0
41.57706093189964
0.0
0.0
0.0
0.0
0.7211538461538461
9.090909090909092
2.964959568733154
6.528189910979229
6.333333333333334
0.0
0.0
0.0
0.0
0.0
0.0
4.15335463258786
100.0
1.1111111111111112
48.40764331210191
84.63611859838275
100.0
0.0
0.0
0.0
76.39484978540773
44.76923076923077
0.0
1.1086474501108647
15.503875968992247
27.77777777777778
24.20212765957447
49.09090909090909
62.054794520547944
90.7871198568873
0.0
57.11060948081265
76.19047619047619
2.147239263803681
1.4354066985645932
3.1578947368421053
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.8901734104046244
17.959183673469386
13.389121338912133
0.0
84.25925925925925
63.46153846153846
52.94117647058824
83.28424153166422
4.417670682730924
0.5154639175257731
8.81542699724518
81.77641653905053
8.74316939

In [212]:
def nlsts_to_mins(wins):
    return [(((a[0]-wins[0][0])/1e6)/60,((a[1]-wins[0][0])/1e6)/60) for a in wins]

### Fist look at competitive interactions between fields

In [64]:
# assemble all fields into array (field id, visit time, rate) where array is sorted by visit time
fields = []
for i,f in enumerate(unit.fields):
    fields.append(np.column_stack((np.asarray([i]*f.shape[0]), f)))
fields = np.vstack([i for i in fields])
fields = fields[np.argsort(fields[:,1])]

In [83]:
with open(df+"sessionEpochInfo.txt","r") as f:
        lines = f.readlines()
startSession_us, endSession_us = int(lines[0].split(',')[0]), int(lines[0].split(',')[1])

cmap = util.makeCustomColormap()

startSession = 0 # because were dealing with relative mins in session, its by def 0
endSession = ((endSession_us-startSession_us)/1e6)/60

wnSize=60*1e6*5
wnStep = 10*1e6

ims = []
start = startSession_us
finished = False

while finished is False:
    end = start + wnSize
    spikes = unit.spikes[np.logical_and(unit.spikes[:,0]>start, unit.spikes[:,0]<=end)]
    pos = unit.position[np.logical_and(unit.position[:,0]>start, unit.position[:,0]<=end)]
    rm = util.makeRM(spikes, pos)
    rm = util.weird_smooth(rm,0.25)
    plt.imshow(rm, aspect='auto', interpolation='None',origin='lower',cmap=cmap,vmin=0,vmax=20)
    plt.pause(0.005)
    plt.clf()
    start = start + wnStep
    if end > endSession_us:
        finished = True 

E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:502: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [10]:
%qtconsole